<a href="https://colab.research.google.com/github/JaviCg1/PySpark/blob/main/PySpark_Notes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Capitulo 2

In [2]:
# Instalar SDK Java 8

!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Descargar Spark 3.4.3

!wget -q https://archive.apache.org/dist/spark/spark-3.4.3/spark-3.4.3-bin-hadoop3.tgz

# Descomprimir el archivo descargado de Spark

!tar xf spark-3.4.3-bin-hadoop3.tgz

# Establecer las variables de entorno

import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.3-bin-hadoop3"

# Instalar la librería findspark

!pip install -q findspark

# Instalar pyspark

!pip install -q pyspark

### verificar la instalación ###

import findspark

findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()

# Probando la sesión de Spark
df = spark.createDataFrame([{"Hola": "Mundo"} for x in range(10)])

df.show(10, False)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
+-----+
|Hola |
+-----+
|Mundo|
|Mundo|
|Mundo|
|Mundo|
|Mundo|
|Mundo|
|Mundo|
|Mundo|
|Mundo|
|Mundo|
+-----+



# Capitulo 3 Inroduccion

In [ ]:
import findspark

findspark.init()

from pyspark.sql import SparkSession

#spark = SparkSession.builder.getOrCreate()

#Se crea la sesion y se le da el numero de cores en master y el nombre que va a tener
spark = SparkSession.builder.master("local[*]").appName('Curso Pyspark').getOrCreate()

spark




In [ ]:
# Diferentes formas de crear un RDD


sc = spark.sparkContext

# Crear un RDD vacío

rdd_vacio = sc.emptyRDD

# Crear un RDD con parallelize

rdd_vacio3 = sc.parallelize([], 3)

rdd_vacio3.getNumPartitions()

rdd = sc.parallelize([1,2,3,4,5])

rdd.collect()

# Crear un RDD desde un archivo de texto



[]

In [ ]:
#Importa y crea un rdd en base a archivos de texto y sus lineas
rdd_texto = sc.textFile('./rdd_source.txt')

rdd_texto.collect()

#A un rdd se le puede añadir un lambda para sumar valores o añadir cosas
rdd_suma = rdd.map(lambda x: x +1)

rdd_suma.collect()

#Aqui creamos un dataframe
df = spark.createDataFrame([(1, 'jose'), (2, 'juan')], ['id', 'nombre'])

df.show()

#Creas un rdd del dataframe
rdd_df = df.rdd

rdd_df.collect()

# Leer el archivo CSV como un DataFrame
df1 = spark.read.csv("rdd_source.txt", header=True, inferSchema=True)

# Mostrar el contenido del DataFrame
df1.show()




+---+------+
| id|nombre|
+---+------+
|  1|  jose|
|  2|  juan|
+---+------+

+---+-------+----+---------+
| id| nombre|edad|   ciudad|
+---+-------+----+---------+
|  1|   Juan|  28|   Madrid|
|  2|  María|  22|Barcelona|
|  3|  Pedro|  35|  Sevilla|
|  4|    Ana|  30| Valencia|
|  5|   Luis|  40|   Bilbao|
|  6|  Sofía|  25|   Málaga|
|  7| Andrés|  33| Zaragoza|
|  8|  Lucía|  29|Santander|
|  9|Roberto|  45|  Granada|
| 10|  Elena|  27| Alicante|
+---+-------+----+---------+



Ejercicios de prueba

In [ ]:
ppp = sc.textFile('./rdd_source.txt')
print("Contenido del RDD:")


print(f"Número de particiones: {ppp.getNumPartitions()}")
ppp.collect()



Contenido del RDD:
Número de particiones: 2


['id,nombre,edad,ciudad',
 '1,Juan,28,Madrid',
 '2,María,22,Barcelona',
 '3,Pedro,35,Sevilla',
 '4,Ana,30,Valencia',
 '5,Luis,40,Bilbao',
 '6,Sofía,25,Málaga',
 '7,Andrés,33,Zaragoza',
 '8,Lucía,29,Santander',
 '9,Roberto,45,Granada',
 '10,Elena,27,Alicante']

# Capitulo 4 Transoformaciones RDD

In [ ]:
# Transformaciones: función map

import findspark
findspark.init()
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

sc = spark.sparkContext





ModuleNotFoundError: No module named 'findspark'

##Map

In [ ]:
rdd = sc.parallelize([1,2,3,4,5])

rdd_resta = rdd.map(lambda x: x - 1)
rdd_resta.collect()

[0, 1, 2, 3, 4]

In [ ]:
rdd_par = rdd.map(lambda x: x % 2 == 0)

rdd_par.collect()



[False, True, False, True, False]

In [ ]:
rdd_texto = sc.parallelize(['jose', 'juan', 'lucia'])

rdd_mayuscula = rdd_texto.map(lambda x: x.upper())

rdd_mayuscula.collect()

rdd_hola = rdd_mayuscula.map(lambda x: 'Hola ' + x)

rdd_hola.collect()

['Hola JOSE', 'Hola JUAN', 'Hola LUCIA']

##FlatMap

In [ ]:
rdd = sc.parallelize([1,2,3,4,5])

#Esta funcion devuelve otro rrd con dos partes: x , x^2
rdd_cuadrado = rdd.map(lambda x: (x, x ** 2))


rdd_cuadrado.collect()
#rdd_cuadrado.getNumPartitions()



[(1, 1), (2, 4), (3, 9), (4, 16), (5, 25)]

In [ ]:
#Transoforma el rdd en un map plano (Transforma los objetos en una lista plana)
rdd_cuadrado_flat = rdd.flatMap(lambda x: (x, x ** 2))

rdd_cuadrado_flat.collect()



[1, 1, 2, 4, 3, 9, 4, 16, 5, 25]

In [ ]:
rdd_texto = sc.parallelize(['jose', 'juan', 'lucia'])

rdd_mayuscula = rdd_texto.flatMap(lambda x: (x, x.upper()))

rdd_mayuscula.collect()

['jose', 'JOSE', 'juan', 'JUAN', 'lucia', 'LUCIA']

## Filter

In [ ]:
rdd = sc.parallelize([1,2,3,4,5,6,7,8,9])

#Con map hubiera devuleto el resultado y aqui devuelve los valores que sean true
rdd_par = rdd.filter(lambda x: x % 2 == 0)

rdd_par.collect()



[2, 4, 6, 8]

In [ ]:
rdd_impar = rdd.filter(lambda x: x % 2 != 0)

rdd_impar.collect()



[1, 3, 5, 7, 9]

In [ ]:
rdd_texto = sc.parallelize(['jose', 'juaquin', 'juan', 'lucia', 'karla', 'katia'])

rdd_k = rdd_texto.filter(lambda x: x.startswith('p'))

rdd_k.collect()



In [ ]:
rdd_filtro = rdd_texto.filter(lambda x: x.startswith('j') and x.find('u') == 1)

rdd_filtro.collect()

['juaquin', 'juan']

##coalesce

In [ ]:
rdd = sc.parallelize([1,2,3.4,5], 10)

rdd.getNumPartitions()
#Junta particiones y las fusiona

rdd1=rdd.coalesce(5)
rdd1.getNumPartitions()
rdd1.collect()

[1, 2, 3.4, 5]

In [ ]:
rdd.getNumPartitions()

rdd5 = rdd.coalesce(5)

rdd5.getNumPartitions()

##Repartition

In [ ]:
#Incrementa el numero de particiones, coalesce es mas optima para reducir el numero
rdd = sc.parallelize([1,2,3,4,5], 3)

rdd.getNumPartitions()

rdd7 = rdd.repartition(7)

rdd7.getNumPartitions()

7

##ReducedByKey

In [ ]:
#Optimiza los datos de las particiones en caso de tener la misma key
rdd = sc.parallelize(
    [('casa', 1),
     ('parque', 1),
     ('que', 5),
     ('casa', 1),
     ('escuela', 2),
     ('casa', 2),
     ('que', 1)]
)

rdd.collect()

#rdd.getNumPartitions()



[('casa', 1),
 ('parque', 1),
 ('que', 5),
 ('casa', 1),
 ('escuela', 2),
 ('casa', 2),
 ('que', 1)]

In [ ]:
rdd_reduciodo = rdd.reduceByKey(lambda x,y: x + y)

rdd_reduciodo.collect()

[('parque', 1), ('que', 6), ('casa', 4), ('escuela', 2)]

##Ejercicios

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

sc = spark.sparkContext

In [ ]:
rdd = sc.parallelize(['piton', 'scala', 'spark'])
rddMayus= rdd.map(lambda x: x.upper())

rddMayus.collect()



['PITON', 'SCALA', 'SPARK']

In [ ]:
rddP = rdd.filter(lambda x: x.startswith('p'))
rddP.collect()

['piton']

In [ ]:
rdd = sc.parallelize([50,51,52,53,54,55,56,57,58,59])
rddRaiz= rdd.flatMap(lambda x: (x, x ** 0.5))
rdd7 = rddRaiz.repartition(7)
rdd7.collect()
rdd7.getNumPartitions()

7

In [ ]:
rdd_trans = sc.textFile('./transacciones')

rdd_redu = rdd_trans.reduceByKey(lambda x,y: x + y)



NameError: name 'sc' is not defined

#Capitulo 5

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

##Reduce

In [ ]:
rdd = sc.parallelize([2,4,6,8])
#reduce el rdd entero y lo manda al drive y este si que hace que se ejecute
rdd.reduce(lambda x,y: x + y)

rdd1 = sc.parallelize([1,2,3,4])

rdd1.reduce(lambda x,y: x * y)

24

##Count

In [ ]:
rdd = sc.parallelize(['j', 'o', 's', 'e'])
#Cuenta
rdd.count()

rdd1 = sc.parallelize([item for item in range(10)])

rdd1.count()

# Elevar al cuadrado los números
# [item ** 2 for item in range(10)]

# Filtrar para obtener solo los números pares
# [item for item in range(10) if item % 2 == 0]

# Filtrar para obtener solo los números impares
# [item for item in range(10) if item % 2 != 0]

# Crear tuplas con el número y su cuadrado
# [(item, item ** 2) for item in range(10)]

# Filtrar para obtener solo números impares mayores que 5
# [item for item in range(10) if item % 2 != 0 and item > 5]

# Aplicar una función a cada número de la lista
# def f(x):
#     return x + 10
# [f(item) for item in range(10)]

# Anidar comprehensions para crear pares de números
# [[x, y] for x in range(3) for y in range(3)]

# Convertir los números a cadenas de texto
# [str(item) for item in range(10)]


10

##Collect

In [ ]:
rdd = sc.parallelize('Hola Apache Spark!'.split(' '))

rdd.collect()

rdd1 = sc.parallelize([(item, item ** 2) for item in range(20)])

rdd1.collect()

[(0, 0),
 (1, 1),
 (2, 4),
 (3, 9),
 (4, 16),
 (5, 25),
 (6, 36),
 (7, 49),
 (8, 64),
 (9, 81),
 (10, 100),
 (11, 121),
 (12, 144),
 (13, 169),
 (14, 196),
 (15, 225),
 (16, 256),
 (17, 289),
 (18, 324),
 (19, 361)]

##Take max y algo mas

In [ ]:
# take

rdd = sc.parallelize('La programación es bella'.split(' '))

rdd.take(2)

rdd.take(4)

# max

rdd1 = sc.parallelize([item/(item + 1) for item in range(10)])

rdd1.max()

rdd1.collect()



[0.0,
 0.5,
 0.6666666666666666,
 0.75,
 0.8,
 0.8333333333333334,
 0.8571428571428571,
 0.875,
 0.8888888888888888,
 0.9]

##Ejercicios

In [ ]:
import math

# Calcula el factorial de 5

rdd = sc.parallelize([item for item in range(100)])
rdd1 = rdd.map(lambda x: (x, math.factorial(x)))
rdd1.collect()


[(0, 1),
 (1, 1),
 (2, 2),
 (3, 6),
 (4, 24),
 (5, 120),
 (6, 720),
 (7, 5040),
 (8, 40320),
 (9, 362880),
 (10, 3628800),
 (11, 39916800),
 (12, 479001600),
 (13, 6227020800),
 (14, 87178291200),
 (15, 1307674368000),
 (16, 20922789888000),
 (17, 355687428096000),
 (18, 6402373705728000),
 (19, 121645100408832000),
 (20, 2432902008176640000),
 (21, 51090942171709440000),
 (22, 1124000727777607680000),
 (23, 25852016738884976640000),
 (24, 620448401733239439360000),
 (25, 15511210043330985984000000),
 (26, 403291461126605635584000000),
 (27, 10888869450418352160768000000),
 (28, 304888344611713860501504000000),
 (29, 8841761993739701954543616000000),
 (30, 265252859812191058636308480000000),
 (31, 8222838654177922817725562880000000),
 (32, 263130836933693530167218012160000000),
 (33, 8683317618811886495518194401280000000),
 (34, 295232799039604140847618609643520000000),
 (35, 10333147966386144929666651337523200000000),
 (36, 371993326789901217467999448150835200000000),
 (37, 1376375309

#Capitulo 6


In [ ]:
# Almacenamiento en caché

import findspark
findspark.init()
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext


In [ ]:
#Se selecciona donde se guarda la informacion si en disco o en memoria. Memoria es si entra es mucho mejor
rdd = sc.parallelize([item for item in range(10)])

from pyspark.storagelevel import StorageLevel

rdd.persist(StorageLevel.MEMORY_ONLY)

rdd.unpersist()

rdd.persist(StorageLevel.DISK_ONLY)

rdd.unpersist()

rdd.cache()

##Organizacion de las particiones y su creacion

In [ ]:
rdd = sc.parallelize(['x', 'y', 'z'])

hola = 'Hola'

hash(hola)

num_particiones = 6

# indice = hash(item) % num_particiones

hash('x') % num_particiones

hash('y') % num_particiones

hash('z') % num_particiones

##Variable Broadcas

In [ ]:
rdd = sc.parallelize([item for item in range(10)])

uno = 1

br_uno = sc.broadcast(uno)

rdd1 = rdd.map(lambda x: x + br_uno.value)

rdd1.collect()

br_uno.unpersist()

rdd1  = rdd.map(lambda x: x + br_uno.value)

rdd1.collect()

br_uno.destroy()

rdd1  = rdd.map(lambda x: x + br_uno.value)

rdd1.take(5)

##Acumuladores

In [ ]:
acumulador = sc.accumulator(0)

rdd = sc.parallelize([2,4,6,8,10])

rdd.foreach(lambda x: acumulador.add(x))

print(acumulador.value)

rdd1 = sc.parallelize('Mi nombre es Jose Miguel y me siento genial'.split(' '))

acumulador1 = sc.accumulator(0)

rdd1.foreach(lambda x: acumulador1.add(1))

print(acumulador1.value)

30
9


#Capitulo 7 DataFrame

In [ ]:
# Creando DataFrames

import findspark
findspark.init()
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [ ]:


rdd = sc.parallelize([item for item in range(10)]).map(lambda x: (x, "x ** 2"))

rdd.collect()

#pasa de rdd a dataframe con dos columnas
df = rdd.toDF(['numero', 'cudrado'])

df.printSchema()

df.show()




root
 |-- numero: long (nullable = true)
 |-- cudrado: string (nullable = true)

+------+-------+
|numero|cudrado|
+------+-------+
|     0| x ** 2|
|     1| x ** 2|
|     2| x ** 2|
|     3| x ** 2|
|     4| x ** 2|
|     5| x ** 2|
|     6| x ** 2|
|     7| x ** 2|
|     8| x ** 2|
|     9| x ** 2|
+------+-------+



##Pasar de rdd a dataframe

In [ ]:
# Crear un DataFrame a partir de un RDD con schema

rdd1 = sc.parallelize([(1, 'Jose', 35.5), (2, 'Teresa', 54.3), (3, 'Katia', 12.7)])


from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType

# Primera vía

esquema1 = StructType(
    [
     StructField('id', IntegerType(), True),
     StructField('nombre', StringType(), True),
     StructField('saldo', DoubleType(), True)
    ]
)

# Segunda vía

esquema2 = "`id` INT, `nombre` STRING, `saldo` DOUBLE"

df1 = spark.createDataFrame(rdd1, schema=esquema1)

df1.printSchema()

df1.show()

df2 = spark.createDataFrame(rdd1, schema=esquema2)

df2.printSchema()

df2.show()

# Crear un DataFrame a partir de un rango de números

spark.range(5).toDF('id').show()

spark.range(3, 15).toDF('id').show()

spark.range(0, 20, 2).toDF('id').show()


##Crear dataframe en base a datos

In [ ]:
#Lee los datos de un txt y crea el df
df = spark.read.text('./data/dataTXT.txt')

df.show()

df.show(truncate=False)

# Crear un DataFrame mediante la lectura de un archivo csv

df1 = spark.read.csv('./data/dataCSV.csv')

df1.show()

#Le decaramos que la primera fila es el nombre de la columna
df1 = spark.read.option('header', 'true').csv('./data/dataCSV.csv')

df1.show()

# Leer un archivo de texto con un delimitador diferente comoo el |

df2 = spark.read.option('header', 'true').option('delimiter', '|').csv('./data/dataTab.txt')

df2.show()

# Crear un DataFrame a partir de un json proporcionando un schema

from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType

json_schema =  StructType(
    [
     StructField('color', StringType(), True),
     StructField('edad', IntegerType(), True),
     StructField('fecha', DateType(), True),
     StructField('pais', StringType(), True)
    ]
)

df4 = spark.read.schema(json_schema).json('./data/dataJSON.json')

df4.show()

df4.printSchema()

# Crear un DataFrame a partir de un archivo parquet

df5 = spark.read.parquet('./data/dataPARQUET.parquet')

df5.show()

# Otra alternativa para leer desde una fuente de datos parquet en este caso

df6 = spark.read.format('parquet').load('./data/dataPARQUET.parquet')

df6.printSchema()

##Columnas

In [ ]:
df = spark.read.parquet('./data/dataPARQUET.parquet')

df.printSchema()

# Primera alternativa para referirnos a las columnas

df.select('title').show()

# Segunda alternativa

from pyspark.sql.functions import col

df.select(col('title')).show()

##Select

In [ ]:
df = spark.read.parquet('./data/datos.parquet')

df.printSchema()

from pyspark.sql.functions import col

df.select(col('video_id')).show()

df.select('video_id', 'trending_date').show()

# Esta vía nos dará error

df.select(
    'likes',
    'dislikes',
    ('likes' - 'dislikes')
).show()

# Forma correcta
#Para realizar tus propias operacioens
df.select(
    col('likes'),
    col('dislikes'),
    (col('likes') - col('dislikes')).alias('aceptacion')
).show()

# selectExpr

df.selectExpr('likes', 'dislikes', '(likes - dislikes) as aceptacion').show()

df.selectExpr("count(distinct(video_id)) as videos").show()

##Filter

In [ ]:
df = spark.read.parquet('./data/datos.parquet')

# filter

from pyspark.sql.functions import col

df.show()
#Aparezca solo lass filas  videos id con ese id
df.filter(col('video_id') == '2kyS6SvSYSE').show()

#Muy parecido a filter el where
df1 = spark.read.parquet('./data/datos.parquet').where(col('trending_date') != '17.14.11')

df1.show()

df2 = spark.read.parquet('./data/datos.parquet').where(col('likes') > 5000)

df2.filter((col('trending_date') != '17.14.11') & (col('likes') > 7000)).show()

df2.filter(col('trending_date') != '17.14.11').filter(col('likes') > 7000).show()

##DropDuplicates y distinct

In [ ]:
df = spark.read.parquet('./data')

# distinct
#Elimina duplicados
df_sin_duplicados = df.distinct()

print('El conteo del dataframe original es {}'.format(df.count()))
print('El conteo del dataframe sin duplicados es {}'.format(df_sin_duplicados.count()))

# función dropDuplicates

dataframe = spark.createDataFrame([(1, 'azul', 567), (2, 'rojo', 487), (1, 'azul', 345), (2, 'verde', 783)]).toDF('id', 'color', 'importe')

dataframe.show()

#Seleccionas que se eliminen los duplicados en caso de una columna en concreto
dataframe.dropDuplicates(['id', 'color']).show()

##Sort y orderby

In [ ]:
from pyspark.sql.functions import col

df = (spark.read.parquet('./data')
    .select(col('likes'), col('views'), col('video_id'), col('dislikes'))
    .dropDuplicates(['video_id'])
)

df.show()

# sort

df.sort('likes').show()

from pyspark.sql.functions import desc

df.sort(desc('likes')).show()

# función orderBy

df.orderBy(col('views')).show()

df.orderBy(col('views').desc()).show()

dataframe = spark.createDataFrame([(1, 'azul', 568), (2, 'rojo', 235), (1, 'azul', 456), (2, 'azul', 783)]).toDF('id', 'color', 'importe')

dataframe.show()


##Ordena bajo las dos culumnas con la primera con prioridad
dataframe.orderBy(col('color').desc(), col('importe')).show()

# funcion limit, limita la cantidad de resultados

top_10 = df.orderBy(col('views').desc()).limit(10)

top_10.show()

##Withcolum me lo salte

##Drop sample y random split

In [ ]:
df = spark.read.parquet('./data')

# drop, elimina columnas

df.printSchema()

df_util = df.drop('comments_disabled')

df_util.printSchema()

df_util = df.drop('comments_disabled', 'ratings_disabled', 'thumbnail_link')

df_util.printSchema()

df_util = df.drop('comments_disabled', 'ratings_disabled', 'thumbnail_link', 'cafe')

df_util.printSchema()

# sample

df_muestra = df.sample(0.8)

num_filas = df.count()
num_filas_muestra = df_muestra.count()

print('El 80% de filas del dataframe original es {}'.format(num_filas - (num_filas*0.2)))
print('El numero de filas del dataframe muestra es {}'.format(num_filas_muestra))

df_muestra = df.sample(fraction=0.8, seed=1234)

df_muestra = df.sample(withReplacement=True, fraction=0.8, seed=1234)

# randomSplit

train, test = df.randomSplit([0.8, 0.2], seed=1234)

train, validation, test = df.randomSplit([0.6, 0.2, 0.2], seed=1234)

train.count()

validation.count()

test.count()

##Datos faltantes o incorrectos

In [ ]:
df = spark.read.parquet('./data/')

df.count()
#Elimina las filas que tienen un valor null
df.na.drop().count()

df.na.drop('any').count()

df.dropna().count()

df.na.drop(subset=['views']).count()

df.na.drop(subset=['views', 'dislikes']).count()

from pyspark.sql.functions import col

df.orderBy(col('views')).select(col('views'), col('likes'), col('dislikes')).show()

df.fillna(0).orderBy(col('views')).select(col('views'), col('likes'), col('dislikes')).show()

df.fillna(0, subset=['likes', 'dislikes']).orderBy(col('views')).select(col('views'), col('likes'), col('dislikes')).show()

##Spark

In [ ]:
df = spark.read.parquet('./data/')

# show

df.show()

df.show(5)

#hace que se vean las lineas enteras
df.show(5, truncate=False)

# take, te da la fila directamente

df.take(1)

# head

df.head(1)

# collect

df.select('likes').collect()

##Escritura de Dataframes

In [ ]:
df = spark.read.parquet('./data/')
#Guardas el df en un archivo .csv y se cra un archivo por cada una de las particiones que tenga dl df
df1 = df.repartition(2)

df1.write.format('csv').option('sep', '|').save()

df1.coalesce(1).write.format('csv').option('sep', '|').save('./output/csv1')

df.printSchema()
##Enseña todas las lineas diferentes quqe hay en commets disabled
#En este caso para comprobar por que solo tendria que haber true or false
df.select('comments_disabled').distinct().show()

from pyspark.sql.functions import col
#Solo se mantienen las lineas que comments.. tengan true or false
df_limpio = df.filter(col('comments_disabled').isin('True', 'False'))

df_limpio.write.partitionBy('comments_disabled').parquet('./output/parquet')

##Buscar en internet conexion y creacion de un bucket en aws o gcs o azure blob

##Persistir

In [ ]:
df = spark.createDataFrame([(1, 'a'), (2, 'b'), (3, 'c')], ['id', 'valor'])

df.show()
## hace que los cambios que vayas haciendo se van guardando
df.persist()

df.unpersist()

df.cache()

from pyspark.storagelevel import StorageLevel

df.persist(StorageLevel.DISK_ONLY)

df.persist(StorageLevel.MEMORY_AND_DISK)

##Ejercicios

In [ ]:
df = (spark.read.parquet('./data')
    .select(col('likes'), col('views'), col('video_id'), col('dislikes'))
    .dropDuplicates(['video_id'])
)

df.show()

# sort

df.select('likes','dislikes').sort('ciudades').show(3)




df_sin_duplicados = df.distinct()
df.filter(col('infected_id') == 'true' | col('infected_id') == 'false').count().show()

df.filter(col('fem') =='fem').selct().show...

#Capitulo 8 No se de que va PRO

In [ ]:
#Aqui ya se ejecutan ocn funciones mas complejas

import findspark
findspark.init()
from pyspark.sql import SparkSession


spark = SparkSession.builder.getOrCreate()



##Funciones count, countDistinct y approx_count_distinct

In [ ]:
# Funciones count, countDistinct y approx_count_distinct

# count
df = spark.read.parquet('./data/dataframe')

df.printSchema()

df.show()

from pyspark.sql.functions import count

#Crea un df solo de los dos valores de count de colores y nombres sin contar nulos

df.select(
    count('nombre').alias('conteo_nombre'),
    count('color').alias('conteo_color')
).show()

# realiza * un conteo de todas las filas
df.select(
    count('nombre').alias('conteo_nombre'),
    count('color').alias('conteo_color'),
    count('*').alias('conteo_general')
).show()

# countDistinct

from pyspark.sql.functions import countDistinct

df.select(
    countDistinct('color').alias('colores_dif')
).show()

# approx_count_distinct, da una aproximacion por que una exacta podria requerir mucha carga  de trabajo en conjunto datos grandes

from pyspark.sql.functions import approx_count_distinct

dataframe = spark.read.parquet('./data/vuelos')

dataframe.printSchema()

dataframe.select(
    countDistinct('AIRLINE'),
    approx_count_distinct('AIRLINE')
).show()


##Funciones min y max

In [ ]:
from pyspark.sql.functions import min, max, col

vuelos.select(
    min('AIR_TIME').alias('menor_timepo'),
    max('AIR_TIME').alias('mayor_tiempo')
).show()

vuelos.select(
    min('AIRLINE_DELAY'),
    max('AIRLINE_DELAY')
).show()

##Funciones sum, sumDistinct y avg

In [ ]:
# Funciones sum, sumDistinct y avg

from pyspark.sql.functions import sum, sumDistinct, avg, count

# sum

vuelos.printSchema()

vuelos.select(
    sum('DISTANCE').alias('sum_dis')
).show()

# sumDistinct

vuelos.select(
    sumDistinct('DISTANCE').alias('sum_dis_dif')
).show()

# avg

vuelos.select(
    avg('AIR_TIME').alias('promedio_aire'),
    (sum('AIR_TIME') / count('AIR_TIME')).alias('prom_manual')
).show()



##Agregación con agrupación

In [ ]:
# Agregación con agrupación



vuelos = spark.read.parquet('./data/')

vuelos.printSchema()

# Argupa cada aeropuerto y le hace un conteo de cada uno de ellos
from pyspark.sql.functions import desc

(vuelos.groupBy('ORIGIN_AIRPORT')
    .count()
    .orderBy(desc('count'))
).show()

(vuelos.groupBy('ORIGIN_AIRPORT', 'DESTINATION_AIRPORT')
    .count()
    .orderBy(desc('count'))
).show()


##Varias agregaciones por grupo IMPORTANTE

In [ ]:
vuelos = spark.read.parquet('./data/')

from pyspark.sql.functions import count, min, max, desc, avg

vuelos.groupBy('ORIGIN_AIRPORT').agg(
    count('AIR_TIME').alias('tiempo_aire'),
    min('AIR_TIME').alias('min'),
    max('AIR_TIME').alias('max')
).orderBy(desc('tiempo_aire')).show()

vuelos.groupBy('MONTH').agg(
    count('ARRIVAL_DELAY').alias('conteo_de_retrasos'),
    avg('DISTANCE').alias('prom_dist')
).orderBy(desc('conteo_de_retrasos')).show()

##Agregaciones con pivote

In [ ]:
from pyspark.sql.functions import min, max, avg, col

estudiantes.groupBy('graduacion').pivot('sexo').agg(avg('peso')).show()

estudiantes.groupBy('graduacion').pivot('sexo').agg(avg('peso'), min('peso'), max('peso')).show()

estudiantes.groupBy('graduacion').pivot('sexo', ['M']).agg(avg('peso'), min('peso'), max('peso')).show()

estudiantes.groupBy('graduacion').pivot('sexo', ['F']).agg(avg('peso'), min('peso'), max('peso')).show()

##Join de tablas

###inner join

In [ ]:
##iner join
from pyspark.sql.functions import col

join_df = empleados.join(departamentos, col('num_dpto') == col('id'))

join_df.show()

join_df = empleados.join(departamentos, col('num_dpto') == col('id'), 'inner')

join_df.show()

join_df = empleados.join(departamentos).where(col('num_dpto') == col('id'))

join_df.show()

###Left Outer Join y anti join

In [ ]:
from pyspark.sql.functions import col

empleados.join(departamentos, col('num_dpto') == col('id'), 'leftouter').show()

empleados.join(departamentos, col('num_dpto') == col('id'), 'left_outer').show()

empleados.join(departamentos, col('num_dpto') == col('id'), 'left').show()


#El anti join es lo mismo que el outer join de cada lado pero al contrario
empleados.join(departamentos, col('num_dpto') == col('id'), 'outer').show()

#Semi join hace casi lo mismo que el inner join

##Columnas repetidas

In [ ]:
from pyspark.sql.functions import col

depa = departamentos.withColumn('num_dpto', col('id'))

depa.printSchema()

empleados.printSchema()

# Devuelve un error

empleados.join(depa, col('num_dpto') == col('num_dpto'))

# Forma correcta

df_con_duplicados = empleados.join(depa, empleados['num_dpto'] == depa['num_dpto'])

df_con_duplicados.printSchema()

df_con_duplicados.select(empleados['num_dpto']).show()

df2 = empleados.join(depa, 'num_dpto')

df2.printSchema()

empleados.join(depa, ['num_dpto']).printSchema()

##Shuffle has join

In [ ]:
empleados = spark.read.parquet('./data/empleados/')

departamentos = spark.read.parquet('./data/departamentos/')

from pyspark.sql.functions import col, broadcast

empleados.join(broadcast(departamentos), col('num_dpto') == col('id')).show()

empleados.join(broadcast(departamentos), col('num_dpto') == col('id')).explain()

##Ejercicios

In [ ]:
df1 = spark.read.option('header', 'true').csv('./players.csv')


(df1.groupBy('country_of_citizenship').count().orderBy(desc('count'))).show()


+----------------------+-----+
|country_of_citizenship|count|
+----------------------+-----+
|                 Spain| 1949|
|                 Italy| 1884|
|                France| 1758|
|                Brazil| 1621|
|               England| 1585|
|               Ukraine| 1490|
|                Russia| 1486|
|           Netherlands| 1438|
|                Turkey| 1416|
|               Germany| 1333|
|                Greece| 1228|
|              Portugal| 1168|
|               Belgium| 1118|
|               Denmark| 1112|
|              Scotland|  965|
|             Argentina|  598|
|               Türkiye|  585|
|                Serbia|  384|
|                  null|  380|
|               Nigeria|  323|
+----------------------+-----+
only showing top 20 rows



In [ ]:
#dfplayers = spark.read.option('header', 'true').csv('./players.csv')
#dfplayers.printSchema()

dfap = spark.read.option('header', 'true').csv('./appearances.csv')
#dfap.printSchema()
#dfap.show()

dfap.groupBy('player_name').agg(sum('goals').alias('goals'),sum('assists').alias('assists'),sum('minutes_played').alias('minutes_played'),(sum('goals') / sum('minutes_played')).alias('goals_per_minute') ).orderBy(desc('goals')).show()


+--------------------+-----+-------+--------------+--------------------+
|         player_name|goals|assists|minutes_played|    goals_per_minute|
+--------------------+-----+-------+--------------+--------------------+
|  Robert Lewandowski|471.0|  116.0|       48131.0|0.009785792940100974|
|        Lionel Messi|451.0|  224.0|       44380.0|0.010162235241099594|
|   Cristiano Ronaldo|432.0|  114.0|       41150.0|0.010498177399756987|
|         Luis Suárez|316.0|  160.0|       43629.0|0.007242888904169245|
|          Harry Kane|314.0|   75.0|       37076.0|0.008469090516776351|
|Pierre-Emerick Au...|294.0|   80.0|       38613.0|0.007614016004972419|
|       Mohamed Salah|264.0|  125.0|       38183.0|0.006914071707304...|
|       Ciro Immobile|259.0|   67.0|       34661.0|0.007472375292115058|
|       Karim Benzema|257.0|  122.0|       35515.0|0.007236378994790...|
|       Romelu Lukaku|257.0|   77.0|       38528.0|0.006670473421926911|
|       Kylian Mbappé|244.0|  101.0|       24866.0|

In [ ]:
dpp = spark.read.option('header', 'true').csv('./clubs.csv')
dpp.show()

+-------+-------------------+--------------------+-----------------------+------------------+----------+-----------+-----------------+---------------------+---------------------+--------------------+-------------+-------------------+----------+-----------+--------------------+--------------------+
|club_id|          club_code|                name|domestic_competition_id|total_market_value|squad_size|average_age|foreigners_number|foreigners_percentage|national_team_players|        stadium_name|stadium_seats|net_transfer_record|coach_name|last_season|            filename|                 url|
+-------+-------------------+--------------------+-----------------------+------------------+----------+-----------+-----------------+---------------------+---------------------+--------------------+-------------+-------------------+----------+-----------+--------------------+--------------------+
|    105|    sv-darmstadt-98|     SV Darmstadt 98|                     L1|              null|        27

In [ ]:
from pyspark.sql.functions import sum, sumDistinct, avg, count, desc, col

#dfap1.show()
#dfap1.groupBy('player_name').agg(sum('red_cards').alias('red_cards')).orderBy(desc('red_cards')).show()

#dfap1.join(dpp, col('player_club_id') == col('club_id')).groupBy('player_id', 'player_club_id').agg(sum('red_cards').alias('red_cards')).select(col('player_name'),col('club_code'),col('red_cards')).orderBy(desc('red_cards')).show()
#dfap1.join(dpp, col('player_club_id') == col('club_id')).show()
# sum. .select(col('player_name'), col('club_code'))

from pyspark.sql import functions as F


dfap1.join(dpp, col('player_club_id') == col('club_id')).groupBy('player_id', 'player_club_id', 'player_name', 'name').agg(sum('red_cards').alias('red_cards')).select(col('red_cards'),col('name'),col('player_name')).orderBy(desc('red_cards')).show()





+---------+--------------------+---------------+
|red_cards|                name|    player_name|
+---------+--------------------+---------------+
|      4.0|Real Betis Balomp...|Germán Pezzella|
|      4.0|Getafe Club de Fú...|          Djené|
|      3.0|Real Sociedad de ...|Aritz Elustondo|
|      3.0|Club Atlético de ...|   Ángel Correa|
|      3.0|Getafe Club de Fú...|  Damián Suárez|
|      3.0|Getafe Club de Fú...|   Alexis Ruano|
|      3.0|Real Club Celta d...|   Rubén Blanco|
|      3.0|           Málaga CF|           Duda|
|      3.0|Sevilla Fútbol Cl...|   Marcos Acuña|
|      3.0|Real Betis Balomp...|    Luiz Felipe|
|      3.0|Reial Club Deport...|    Sergi Gómez|
|      3.0|Club Atlético Osa...|    Chimy Ávila|
|      3.0|Real Betis Balomp...|    Nabil Fekir|
|      3.0|            Cádiz CF|   Iza Carcelén|
|      3.0|Getafe Club de Fú...|     Allan Nyom|
|      2.0|Valencia Club de ...|          Jonas|
|      2.0|          Levante UD|  David Navarro|
|      2.0|Athletic 

In [ ]:
dfap1.join(dpp, dfap1.player_club_id == dpp.club_id) \
    .groupBy('player_id', 'player_club_id', 'player_name', 'club_code') \
    .agg(F.sum('red_cards').alias('red_cards')) \
    .orderBy(F.desc('red_cards')) \
    .select('player_name', 'club_code', 'red_cards') \
    .show()

In [ ]:
from pyspark.sql.functions import min, max, col, count
dfjuegos = spark.read.option('header', 'true').csv('./games.csv')
#dfjuegos.show()

dfjuegos.select(
    min('season'),
    max('season'),
).show()

dfjuegos.groupBy('competition_id').agg(
    count('game_id').alias('partidos')).show()

+-----------+-----------+
|min(season)|max(season)|
+-----------+-----------+
|       2012|       2024|
+-----------+-----------+

+--------------+--------+
|competition_id|partidos|
+--------------+--------+
|           CLQ|    1140|
|           SUC|      28|
|           IT1|    4590|
|           CGB|     241|
|           BE1|    3115|
|          POSU|      12|
|          UCOL|     141|
|          FRCH|      12|
|           SFA|    1134|
|           CIT|     864|
|          KLUB|      93|
|           RUP|    1410|
|            CL|    1494|
|           USC|      13|
|          UKRP|     587|
|           NLP|    1030|
|           DFL|      13|
|           SCI|      14|
|           GR1|    2742|
|           GRP|    1081|
+--------------+--------+
only showing top 20 rows



In [ ]:
dpp = spark.read.option('header', 'true').csv('./clubs.csv')
dpp.show()

+-------+-------------------+--------------------+-----------------------+------------------+----------+-----------+-----------------+---------------------+---------------------+--------------------+-------------+-------------------+----------+-----------+--------------------+--------------------+
|club_id|          club_code|                name|domestic_competition_id|total_market_value|squad_size|average_age|foreigners_number|foreigners_percentage|national_team_players|        stadium_name|stadium_seats|net_transfer_record|coach_name|last_season|            filename|                 url|
+-------+-------------------+--------------------+-----------------------+------------------+----------+-----------+-----------------+---------------------+---------------------+--------------------+-------------+-------------------+----------+-----------+--------------------+--------------------+
|    105|    sv-darmstadt-98|     SV Darmstadt 98|                     L1|              null|        27

In [ ]:
dfjuegos.groupBy('competition_id').agg(
    sum('attendance').alias('publico_total')).orderBy(desc('publico_total')).show()


+--------------+-------------+
|competition_id|publico_total|
+--------------+-------------+
|           GB1| 1.54710808E8|
|            L1| 1.34250002E8|
|           ES1| 1.13305174E8|
|           IT1|  9.9406514E7|
|           FR1|  8.9093313E7|
|            CL|  6.1081202E7|
|           NL1|   6.033511E7|
|            EL|  4.7293135E7|
|           TR1|  3.5363822E7|
|           PO1|  3.3905343E7|
|           RU1|  3.3556227E7|
|           BE1|  3.0541333E7|
|           SC1|  3.0129945E7|
|           FAC|  2.1558041E7|
|           ELQ|  1.8991402E7|
|           CLQ|  1.5814959E7|
|           DFB|  1.4315035E7|
|           DK1|  1.4192838E7|
|           CDR|  1.2279002E7|
|           GR1|  1.1137597E7|
+--------------+-------------+
only showing top 20 rows



#Capitulo 9


In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

data = spark.read.parquet('./data/convertir')

data.printSchema()

data.show(truncate=False)
from pyspark.sql.functions import col, to_date, to_timestamp

##Fecha y hora

In [ ]:
# Funciones de fecha y hora
# Ayuda para restar fechas...

data1 = data.select(
    to_date(col('date')).alias('date1'),
    to_timestamp(col('timestamp')).alias('ts1'),
    to_date(col('date_str'), 'dd-MM-yyyy').alias('date2'),
    to_timestamp(col('ts_str'), 'dd-MM-yyyy mm:ss').alias('ts2')

)

data1.show(truncate=False)

data1.printSchema()

from pyspark.sql.functions import date_format

data1.select(
    date_format(col('date1'), 'dd-MM-yyyy')
).show()

df = spark.read.parquet('./data/calculo')

df.show()

##resta fechas
from pyspark.sql.functions import datediff, months_between, last_day

df.select(
    col('nombre'),
    datediff(col('fecha_salida'), col('fecha_ingreso')).alias('dias'),
    months_between(col('fecha_salida'), col('fecha_ingreso')).alias('meses'),
    last_day(col('fecha_salida')).alias('ultimo_dia_mes')
).show()


#Añade dias a la fecha
from pyspark.sql.functions import date_add, date_sub

df.select(
    col('nombre'),
    col('fecha_ingreso'),
    date_add(col('fecha_ingreso'), 14).alias('mas_14_dias'),
    date_sub(col('fecha_ingreso'), 1).alias('menos_1_dia')
).show()

from pyspark.sql.functions import year, month, dayofmonth, dayofyear, hour, minute, second

df.select(
    col('baja_sistema'),
    year(col('baja_sistema')),
    month(col('baja_sistema')),
    dayofmonth(col('baja_sistema')),
    dayofyear(col('baja_sistema')),
    hour(col('baja_sistema')),
    minute(col('baja_sistema')),
    second(col('baja_sistema'))
).show()


##Funciones String

In [ ]:
data = spark.read.parquet('./data/')

data.show()
#Trim elimina espacios
from pyspark.sql.functions import ltrim, rtrim, trim

data.select(
    ltrim('nombre').alias('ltrim'),
    rtrim('nombre').alias('rtrim'),
    trim('nombre').alias('trim')
).show()

from pyspark.sql.functions import col, lpad, rpad

#Añade ese simbolo a la izq o dere hasta rellenar el taño dicho. Ejm 8
data.select(
    trim(col('nombre')).alias('trim')
).select(
    lpad(col('trim'), 8, '-').alias('lpad'),
    rpad(col('trim'), 8, '=').alias('rpad')
).show()

df1 = spark.createDataFrame([('Spark', 'es', 'maravilloso')], ['sujeto', 'verbo', 'adjetivo'])

df1.show()

#Un select y luego otro selecct hace que se haga uno dentro de otro
#Concat concatena, initcap es la primera letra de cada palabra en mayuscula
from pyspark.sql.functions import concat_ws, lower, upper, initcap, reverse

df1.select(
    concat_ws(' ', col('sujeto'), col('verbo'), col('adjetivo')).alias('frase')
).select(
    col('frase'),
    lower(col('frase')).alias('minuscula'),
    upper(col('frase')).alias('mayuscula'),
    initcap(col('frase')).alias('initcap'),
    reverse(col('frase')).alias('reversa')
).show()

from pyspark.sql.functions import regexp_replace

df2 = spark.createDataFrame([(' voy a casa por mis llaves',)], ['frase'])

df2.show(truncate=False)

#Donde encuentre voy o por la cambia por ir
df2.select(
    regexp_replace(col('frase'), 'voy|por', 'ir').alias('nueva_frase')
).show(truncate=False)

##Colecciones, listas arrays

In [ ]:
# Funciones para trabajo con colecciones


data = spark.read.parquet('./data/parquet/')

data.show(truncate=False)

data.printSchema()

#Hay un array en una columna, cuenta el tamaño, lo ordena alfabeticamente y busca
from pyspark.sql.functions import col, size, sort_array, array_contains

data.select(
    size(col('tareas')).alias('tamaño'),
    sort_array(col('tareas')).alias('arreglo_ordenado'),
    array_contains(col('tareas'), 'buscar agua').alias('buscar_agua')
).show(truncate=False)

#Reparte cada tarea de la lista en otra

from pyspark.sql.functions import explode

data.select(
    col('dia'),
    explode(col('tareas')).alias('tareas')
).show()

# Formato JSON

json_df_str = spark.read.parquet('./data/JSON')

json_df_str.show(truncate=False)

json_df_str.printSchema()

# Al obtenerlo de un jswon no se parsea bien
#Lo que hay que hacer es crearle una estructura para que se puedan pasar y entender bien los datos

from pyspark.sql.types import StructType, StructField, StringType, ArrayType

schema_json = StructType(
    [
     StructField('dia', StringType(), True),
     StructField('tareas', ArrayType(StringType()), True)
    ]
)

from pyspark.sql.functions import from_json, to_json

json_df = json_df_str.select(
    from_json(col('tareas_str'), schema_json).alias('por_hacer')
)

json_df.printSchema()

#Lo acabamos adaptando para obetner todos los datos de una columna
json_df.select(
    col('por_hacer').getItem('dia'),
    col('por_hacer').getItem('tareas'),
    col('por_hacer').getItem('tareas').getItem(0).alias('primer_tarea')
).show(truncate=False)

json_df.select(
    to_json(col('por_hacer'))
).show(truncate=False)


##Funciones when, coalesce y lit

In [ ]:
# Funciones when, coalesce y lit


data = spark.read.parquet('./data/')

data.show()

from pyspark.sql.functions import col, when, lit, coalesce

data.select(
    col('nombre'),
    when(col('pago') == 1, 'pagado').when(col('pago') == 2, 'sin pagar').otherwise('sin iniciar').alias('pago')
).show()
# cambia los nulos por un texto
data.select(
    coalesce(col('nombre'), lit('sin nombre')).alias('nombre')
).show()


##Funciones definidas por el usuario UDF IMPORTANTE

In [ ]:
# Funciones definidas por el usuario UDF IMPORTANTE

def cubo(n):
    return n * n * n

from pyspark.sql.types import LongType

spark.udf.register('cubo', f_cubo, LongType())

spark.range(1,10).createOrReplaceTempView('df_temp')

spark.sql("SELECT id, cubo(id) AS cubo FROM df_temp").show()

def bienvenida(nombre):
    return ('Hola {}'.format(nombre))

from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

bienvenida_udf = udf(lambda x: bienvenida(x), StringType())

df_nombre = spark.createDataFrame([('Jose',), ('Julia',)], ['nombre'])

df_nombre.show()

from pyspark.sql.functions import col

df_nombre.select(
    col('nombre'),
    bienvenida_udf(col('nombre')).alias('bie_nombre')
).show()

#-----------------------------Opcion que mas me mola -----------------------------------#


@udf(returnType=StringType())
def mayuscula(s):
    return s.upper()

df_nombre.select(
    col('nombre'),
    mayuscula(col('nombre')).alias('may_nombre')
).show()




#-----------------------------Opcion mas rapida con pandas-----------------------------#

import pandas as pd

from pyspark.sql.functions import pandas_udf

def cubo_pandas(a: pd.Series) -> pd.Series:
    return a * a * a

cubo_udf = pandas_udf(cubo_pandas, returnType=LongType())


df = spark.range(5)

df.select(
    col('id'),
    cubo_udf(col('id')).alias('cubo_pandas')
).show()

##Window partition by

In [ ]:
# Funciones de ventana
#--------------Son formas de mostrar los datos particionados o agrupados --------------


df = spark.read.parquet('./data/')

df.show()

from pyspark.sql.window import Window
from pyspark.sql.functions import desc, row_number, rank, dense_rank, col


#---------------------Particiona en base a los departamentos -------
windowSpec = Window.partitionBy('departamento').orderBy(desc('evaluacion'))

# row_number

df.withColumn('row_number', row_number().over(windowSpec)).filter(col('row_number').isin(1,2)).show()

# rank, ranquea cada posicion de cada numero en base a los grupos

df.withColumn('rank', rank().over(windowSpec)).show()

# dense_rank

df.withColumn('dense_rank', dense_rank().over(windowSpec)).show()

# Agregaciones con especificaciones de ventana

windowSpecAgg = Window.partitionBy('departamento')

from pyspark.sql.functions import min, max, avg

(df.withColumn('min', min(col('evaluacion')).over(windowSpecAgg))
.withColumn('max', max(col('evaluacion')).over(windowSpecAgg))
.withColumn('avg', avg(col('evaluacion')).over(windowSpecAgg))
.withColumn('row_number', row_number().over(windowSpec))
 ).show()

## Catalyst Optimizer


In [ ]:
# Catalyst Optimizer



data = spark.read.parquet('./data/')

data.printSchema()

data.show()

from pyspark.sql.functions import col

nuevo_df = (data.filter(col('MONTH').isin(6,7,8))
            .withColumn('dis_tiempo_aire', col('DISTANCE') / col('AIR_TIME'))
).select(
    col('AIRLINE'),
    col('dis_tiempo_aire')
).where(col('AIRLINE').isin('AA', 'DL', 'AS'))

nuevo_df.explain(True)


#Spark web UI

In [1]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder.config('spark.ui.port', '4050')
    .getOrCreate()
)

from google.colab import output

output.serve_kernel_port_as_window(4050, path='/jobs.index.html')

ModuleNotFoundError: No module named 'pyspark'